In [ ]:
import numpy as np

from scipy import signal
from scipy import integrate
from scipy import fftpack

import matplotlib.pyplot as plt

from utils import *

plt.style.use('seaborn-pastel')

## Simulación de edificio de 4 pisos como masas puntuales con rigideces intermedias

In [ ]:
# Edificio de 4 pisos de 20 toneladas cada uno, con rigideces laterales de 57MN/m entre si
k = 57e6 # N/m
m = 20e3 # kg

k1, k2, k3, k4 = k, k, k, k
m1, m2, m3, m4 = m, m, m, m

# Matrices de rigidez y de masa
K = np.array([[k1+k2, -k2, 0, 0],
             [-k2, k2+k3, -k3, 0],
             [0, -k3, k3+k4, -k4],
             [0, 0, -k4, k4]])

M = np.diag((m1, m2, m3, m4))

print(('K = {}').format(K))
print(('M = {}').format(M))

In [ ]:
# Resuelvo autovectores y autovalores
A = np.linalg.inv(M)@K
lamb, eigv = np.linalg.eig(A)
lamb = np.flip(lamb) # menor a mayor
eigv = np.flip(eigv, axis=1)

In [ ]:
# convierto velocidades angulares a frecuencias
w_modes = np.sqrt(lamb)
f_modes = w_modes/2/np.pi
print(('f = {}').format(np.round(f_modes,3)))

In [ ]:
# Matriz de autovectores (columnas)
print("eigv:")
printMatrix(eigv)

In [ ]:
# Matriz de masas modales
M_modal = np.transpose(eigv)@M@eigv
M_modal[M_modal<1e-3] = np.nan
print("M_modal:")
printMatrix(M_modal)

In [ ]:
# Matriz de rigideces modales
K_modal = np.transpose(eigv)@K@eigv
K_modal[K_modal<1e-3] = np.nan
print("K_modal:")
printMatrix(K_modal)

In [ ]:
# Verifico la validez de las matrices obteniendo las fercuencias modales nuevamente, ahora como w_i = sqrt(k_i/m_i)
w_2 = np.zeros(len(w_modes))
for mode in range(len(w_2)):
    w_2[mode] = np.sqrt(K_modal[mode,mode]/M_modal[mode,mode])
f_2 = w_2/np.pi/2
print(('f2 = {}').format(f_2))

In [ ]:
# Normalizo vectores modales
eigv_norm = np.zeros(eigv.shape)
for mode in range(len(w_modes)):
    eigv_norm[:,mode] = eigv[:,mode]/max(abs(eigv[:,mode]))

In [ ]:
# MAC teórico
MAC_theo = get_MAC_matrix(eigv_norm, eigv_norm)
plot_MAC(MAC_theo, 'Greens', 'k')
print('Max value off diagonal: {:.3f}'.format(get_max_off_diagonal(MAC_theo)))

In [ ]:
# Ploteo de modos
plot_modes(eigv_norm)

## Simulación de aceleración con perfil de ruido blanco

In [ ]:
# Aceleracion maxima en funcion del tiempo
def max_accel(t,t_end):
    a = 0.5*(1-np.cos(2*np.pi*t/t_end))
    return a

In [ ]:
# Simulacion de aceleracion en forma de ruido blanco bajo la curva de aceleracion maxima
t_end = 600
delta_t = 1/119
t = np.linspace(0, t_end, int(t_end/delta_t))

R = 0.1*np.random.normal(size=len(t)) # Vector de ruido gaussiano
R_mean = np.mean(R)

accel = (R - R_mean)*max_accel(t,t_end) # Aceleracion escalada con ruido
max_accel_vec = max_accel(t,t_end) # Vector con aceleraciones puras

In [ ]:
fig, ax = plt.subplots(figsize=(8, 4))
fig.add_axes()
ax.plot(t, accel, color='k', marker='')
ax.plot(t, max_accel_vec, color='r')
ax.grid(True, markevery=1)

In [ ]:
# PSD de aceleraciones en la base
nperseg=450
freq, psd = signal.welch(accel, 
                      fs=1./(delta_t), # sample rate
                      window='hamming', # apply a Hanning window before taking the DFT
                      nperseg=nperseg, # compute periodograms of 256-long segments of x
                      noverlap=nperseg//2,
                      detrend='constant',
                      return_onesided=False) #'constant') # detrend x by subtracting the mean)
plt.figure()
plt.semilogy(freq[1:len(psd)//2], psd[1:len(psd)//2])
plt.ylim([np.min(psd[1:]), np.max(psd[1:])])
plt.xlabel('frequency [Hz]')
plt.ylabel('ASD [g²/Hz]')
plt.grid()
plt.show()
print(len(psd))

In [ ]:
print('ASD promedio: {:.4f} g²/Hz'.format(np.mean(psd)))
print('Grms para ASD constante = ASD*f_max = {:.3f} g_rms'.format(np.mean(psd)*freq[-1]))

In [ ]:
# calculo de g_rms por integracion
area = integrate.simps(psd, freq)
g_rms = np.sqrt(area)
print('Grms = {:.3f} g_rms'.format(g_rms))

In [ ]:
Ug_fft = np.fft.fft(accel) * delta_t
Ug_fft_freq = np.fft.fftfreq(t.shape[-1], d=delta_t)

plt.figure()
plt.semilogy(Ug_fft_freq[:len(Ug_fft_freq)//2], abs(Ug_fft[:len(Ug_fft_freq)//2]))
plt.title('Aceleracion en la base')
plt.xlabel('Frecuencia [Hz]')
plt.ylabel('Espectro de aceleraciones [g/Hz]')

## Simulación de respuesta dinámica del edificio a ruido blanco su base

In [ ]:
xi = 0.025 # fracción de amortiguamiento crítico
Xi = xi*np.ones(eigv_norm.shape[1])
r = np.array([1, 1, 1, 1]).reshape(4,1) # vector logico de desplazamientos respecto de la base
print('xi = {}'.format(Xi[:]))
print('r = {}^T'.format(r[:,0]))

La solucion del espectro de desplazamientos para cada modo es
<br/>
<center> $Y(\omega) = \frac{\frac{\iota}{m_i} U_g(\omega)}{\omega_i^2 - \omega^2 + 2i \xi_i \omega_i \omega}$ </center>
<br/>
con
<center> $\iota = \Phi^T M r$. </center>
<br/>
Y la aceleración es
<center> $\ddot{Y}(\omega) = \omega^2 Y(\omega)$ </center>

In [ ]:
U_g = np.copy(Ug_fft)
w = 2*np.pi*Ug_fft_freq

In [ ]:
I = np.transpose(eigv)@M@r
print(I)
m_modal = M_modal.diagonal() 
Y = np.zeros((len(w), len(w_modes)), dtype=complex)
ddotY = np.copy(Y) 
for mode in range(Y.shape[1]):
    C = I[mode] / (m_modal[mode]*(w_modes[mode]**2 - w**2 + 2*1j*Xi[mode]*w_modes[mode]*w))
    Y[:,mode] = C*Ug_fft
    ddotY[:,mode] = -(w**2)*Y[:,mode]

In [ ]:
f = w/2/np.pi
plt.figure()
lgnd  = ['modo {}'.format(mode+1) for mode in range(Y.shape[1])]
plt.semilogy(f[:len(f)//4], abs(Y[:len(f)//4]))
plt.title('Desplazamientos modales')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Espectro de desplazamientos [m²/Hz]')
plt.ylim([abs(Y[len(f)//4, 0]), None])
plt.legend(lgnd)
plt.show()

In [ ]:
plt.figure()
lgnd  = ['modo {}'.format(mode+1) for mode in range(Y.shape[1])]
plt.semilogy(f[:len(f)//4], abs(ddotY[:len(f)//4]))
plt.title('Aceleraciones modales')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Espectro de Aceleraciones [g/Hz]')
plt.ylim([abs(ddotY[len(f)//4, 0]), None])
plt.legend(lgnd)
plt.show()
print(np.max(abs(ddotY)))

In [ ]:
freq_max = [abs(f[i]) for i in ddotY.argmax(axis=0)]
print(('freqs from max response = {}').format(np.round(freq_max, 3)))
print('errors in % = {}'.format(np.round(100*(f_modes - freq_max) / f_modes, 2)))

### Respuesta en cada piso

In [ ]:
# Espectro de respuesta en frecuencia
X = np.copy(Y)
X = (eigv @ Y.T).T # Desplazamientos en los grados de libertad
ddotX = -(w**2 * X.T).T # Aceleraciones

In [ ]:
plt.figure()
lgnd  = ['piso {}'.format(mode+1) for mode in range(X.shape[1])]
plt.semilogy(f[:len(f)//4], abs(X[:len(f)//4,:]))
plt.title('Desplazamientos por piso')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Espectro de desplazamientos [m²/Hz]')
plt.ylim([abs(X[len(f)//4, 0]), None])
plt.legend(lgnd)
plt.show()

In [ ]:
plt.figure()
lgnd  = ['piso {}'.format(mode+1) for mode in range(X.shape[1])]
plt.semilogy(f[:len(f)//4], abs(ddotX[:len(f)//4,:]))
plt.title('Desplazamientos por piso')
plt.xlabel('Frequency [Hz]')
plt.ylabel('Espectro de desplazamientos [m²/Hz]')
plt.ylim([abs(ddotX[len(f)//4, 0]), None])
plt.legend(lgnd)
plt.show()

## Peak-picking (PP)

In [ ]:
ddotX_psd = np.copy(ddotX)
for piso in range(X.shape[1]):
    ddotX_psd[:, piso] = ddotX[:, piso] * np.conj(ddotX[:, piso])

In [ ]:
# Average Normalized PSD (ANPSD)
ANPSD = np.zeros(ddotX_psd.shape[0], dtype=complex)
NPSD = np.copy(ddotX_psd)
for piso in range(ddotX.shape[1]):
    NPSD[:, piso] = ddotX_psd[:, piso] / np.sum(ddotX_psd[:, piso])
ANPSD = np.sum(NPSD, axis=1)
mode_ind_pp = np.array([m for m in signal.argrelmax(ANPSD[:len(f)//4], order=1000)]).flatten()

In [ ]:
plt.figure()
plt.semilogy(f[:len(f)//4], abs(ANPSD[:len(f)//4]), label='ANPSD')
plt.scatter(f[mode_ind_pp], abs(ANPSD[mode_ind_pp]), label='modes', color='r')
plt.title('Average Normalized PSD')
# plt.xlim([None, 30])
plt.ylim([abs(ANPSD[len(f)//4]), None])
plt.xlabel('Frequency [Hz]')
plt.ylabel('Espectro de desplazamientos [m²/Hz]')
plt.legend()
plt.show()

In [ ]:
print('Frecuencias modales segun Peak-picking')
for mode in range(len(w_modes)):
    idx = mode_ind_pp[mode]
    print('f_{} = {:.3f} Hz ({:.2f}% error)'.format(mode,
                                             f[idx],
                                             100*(f[idx] - f_modes[mode])/ f_modes[mode]))

### Formas modales

In [ ]:
modes_pp = np.copy(eigv) 
for mode in range(modes_pp.shape[1]):
    idx = mode_ind_pp[mode]
    for dof in range(X.shape[1]):
        cross_power = ddotX[idx, dof] * np.conj(ddotX[idx, 0])
        ang = abs(np.angle(cross_power, deg=True))
        if 0 <= ang <= 60:
            sign = 1
        elif 120 <= ang <=180:
            sign = -1
        else:
            sign = 0
        modes_pp[dof, mode] = sign * abs(ddotX[idx, dof]) / abs(ddotX[idx, 0])
        
# Normalizacion
for col in range(modes_pp.shape[1]):
    modes_pp[:,col] = modes_pp[:,col]/max(abs(modes_pp[:,col]))

In [ ]:
# MAC para Peak-picking
MAC_pp = get_MAC_matrix(eigv_norm, modes_pp)
plot_MAC(MAC_pp, 'Greens', 'k')
print('Max value off diagonal: {:.3f}'.format(get_max_off_diagonal(MAC_pp)))

## Frequency Domain Decomposition (FDD)

In [ ]:
# Respuesta en el tiempo
ddotX_time = fftpack.ifft(ddotX, axis=0) # Obtengo respuestas en el tiempo mediante transformada inversa

# Matriz de densidades espectrales cruzadas de la respuesta.
csd_nperseg = 1024
S_xx = np.zeros((csd_nperseg, ddotX_time.shape[1], ddotX_time.shape[1]), dtype=complex)
for piso1 in range(ddotX.shape[1]):
    for piso2 in range (ddotX.shape[1]):
        f_fdd, S_xx[:, piso1, piso2] = signal.csd(ddotX_time[:, piso1], ddotX_time[:, piso2],
                                               fs=1./(delta_t),
                                               window='hanning',
                                               nperseg=csd_nperseg,
                                               detrend='constant',
                                               axis=0,
                                               return_onesided=False)

In [ ]:
# Extraigo frecuencias y formas de modo por SVD
u, s, vh = np.linalg.svd(S_xx)
    
# Maximos del primer valor singular
mode_ind_fdd = np.array([m for m in signal.argrelmax(s[:, 0], order=15)]).flatten()

In [ ]:
plt.semilogy(f_fdd[:len(f_fdd)//4], s[:len(f_fdd)//4, :])
plt.ylim([s[len(f_fdd)//4, 0], None])
plt.scatter(f_fdd[mode_ind_fdd[:len(mode_ind_fdd)//4]],
            abs(s[mode_ind_fdd[:len(mode_ind_fdd)//4], 0]),
            color='r')

In [ ]:
f_fdd[mode_ind_fdd[:4]]

In [ ]:
# Formas modales
modes_fdd = np.zeros(eigv.shape, dtype=complex)
for idx in range(modes_fdd.shape[1]):
    modes_fdd[:, idx] = u[mode_ind_fdd[idx], :, 0]

# Normalizo
for col in range(modes_fdd.shape[1]):
    modes_fdd[:,col] = modes_fdd[:,col]/max(abs(modes_fdd[:,col]))

In [ ]:
# MAC para FDD
MAC_fdd = get_MAC_matrix(eigv_norm, modes_fdd)
plot_MAC(MAC_fdd, 'Greens', 'k')
print('Max value off diagonal: {:.3f}'.format(get_max_off_diagonal(MAC_fdd)))

### Estimación de S_xx

In [ ]:
peak_selec = 1 # nro de pico a estimar
peak_width = 30 # ancho de banda a estimar (en indices)
idxes = [mode_ind_fdd[peak_selec]- peak_width//2, mode_ind_fdd[peak_selec] + peak_width//2]
ff_s = f_fdd[idxes[0]:idxes[1]]
n_modes = 2 # cantidad de modos considerados

In [ ]:
H_shape = (S_xx.shape[1], S_xx.shape[1]*len(ff_s))
H = np.zeros(H_shape)
H = np.reshape(S_xx[idxes[0]:idxes[1], :, :], H_shape, 'F')

w_fdd = 2*np.pi*f_fdd
w_s = 2*np.pi*ff_s

D = np.zeros((len(ff_s), n_modes, n_modes), dtype=complex)
for mode in range(D.shape[1]):
    lamb = w_fdd[mode_ind_fdd[peak_selec]]*(-Xi[mode] + 1j*np.sqrt(1-Xi[mode]**2))
    D[:, mode, mode] = 1/(1j*w_s - lamb)
    
modes_fdd_3d = np.repeat(modes_fdd.T[np.newaxis, :n_modes, :], len(w_s), axis=0)
pre_M_fdd = np.matmul(D, modes_fdd_3d, dtype=complex)
dofs = pre_M_fdd.shape[2]
M_fdd = np.empty_like((pre_M_fdd.reshape((pre_M_fdd.shape[1], pre_M_fdd.shape[0]*dofs),order='C')))
i=0
while i < pre_M_fdd.shape[0]:
    j = i*dofs
    M_fdd[:,j:j+dofs] = pre_M_fdd[i,:,:]
    i += 1
    
tau = np.matmul(H,np.linalg.pinv(M_fdd))

S_xx_hat = np.repeat(tau[np.newaxis, :, :], len(w_s), axis=0)@D@modes_fdd_3d

In [ ]:
plt.semilogy(f_fdd[:len(f_fdd)//4], abs(S_xx[:len(f_fdd)//4,0,0]))
plt.semilogy(ff_s, abs(S_xx_hat[:,0,0]))

In [ ]:
plt.semilogy(ff_s, abs(S_xx[idxes[0]:idxes[1],1,0]))
plt.semilogy(ff_s, abs(S_xx_hat[:,1,0]))

## Enhanced Frequency Domain Decomposition (EFDD)

### Mode PSD identification 

In [ ]:
mac_th = 0.9
total_mode_num = modes_fdd.shape[0]
sdof_freq = np.zeros_like(s, dtype=complex)
sdof_time = np.zeros_like(s, dtype=complex)
t_sdof_time = np.linspace(0, sdof_freq.shape[0]*delta_t, sdof_freq.shape[0])

fig_psd, ax_psd = plt.subplots()
fig_decay, ax_decay = plt.subplots(total_mode_num//2, 2)
ax_psd.grid(True, markevery=1)
[ax_decay.flat[i].grid(True, markevery=1) for i in range(len(ax_decay.flat))]

for mode in range(total_mode_num):
    peak_idx = mode_ind_fdd[mode]
    idx_low, idx_high = get_efdd_segment(u, peak_idx, mac_th)
  
    ax_psd.semilogy(f_fdd[idx_low:idx_high], abs(s[idx_low:idx_high, 0]))
    
    sdof_freq[idx_low:idx_high, mode] = s[idx_low:idx_high, 0]
    sdof_time[:, mode] = fftpack.ifft(sdof_freq[:, mode])
    ax_decay.flat[mode].plot(t_sdof_time[:len(t_sdof_time)//2],
                             sdof_time[:len(t_sdof_time)//2, mode].real)

### Damping estimation

In [ ]:
damp = np.zeros(total_mode_num)
R2 = np.zeros(total_mode_num)

for mode in range(total_mode_num):
    decay = sdof_time[:, mode].real[:len(t_sdof_time)//2]
    damp[mode], R2[mode] = get_damp_from_decay(decay)
    print(f'damp({mode}) = {damp[mode]:.3f}, R2({mode}) = {R2[mode]:.3f}')

### Frequency estimation

In [ ]:
efdd_freq = np.zeros(total_mode_num)

fig_decay, ax_decay = plt.subplots(total_mode_num//2, 2)
[ax_decay.flat[i].grid(True, markevery=1) for i in range(len(ax_decay.flat))]

for mode in range(total_mode_num):
    decay = sdof_time[:, mode].real[:len(t_sdof_time)//2]
    efdd_freq_damped = get_freq_from_signal(t_sdof_time, sdof_time[:, mode])
    efdd_freq[mode] = efdd_freq_damped / np.sqrt(1-damp[mode]**2)
    w_n_efdd = 2*np.pi*efdd_freq[mode]
    peak_ind = np.array([m for m in signal.argrelmax(abs(decay), order=1)]).flatten()

    ax_decay.flat[mode].plot(t_sdof_time[:len(sdof_time)//2], decay)
    ax_decay.flat[mode].plot(t_sdof_time[peak_ind],
                             decay[0]*np.exp(-damp[mode]*w_n_efdd*t_sdof_time[peak_ind]),
                             color='g')

In [ ]:
print('Theo = [',
      ' '.join(['{:.3f}'.format(f_modes[mode]) for mode in range(total_mode_num)]),
      end='] Hz.\n')
print('FDD = [',
      ' '.join(['{:.3f}'.format(f_fdd[mode_ind_fdd[mode]]) for mode in range(total_mode_num)]),
      end='] Hz.\n')
print('EFDD = [',
      ' '.join(['{:.3f}'.format(efdd_freq[mode]) for mode in range(total_mode_num)]),
      end='] Hz.')